# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install geoviews 


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json


# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

city_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    size='Humidity',
    color='City',
    alpha=0.7,
    tiles='OSM',  # Optional: OpenStreetMap background
    title="City Map with Humidity Represented by Point Size"
)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 15) &
    (city_data_df['Max Temp'] <= 25) &
    (city_data_df['Humidity'] < 60) &
    (city_data_df['Cloudiness'] < 10)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
97,97,shenjiamen,29.9576,122.2980,16.43,59,1,7.40,CN,1666108292
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
63,banda,IN,25.4833,80.3333,52,
97,shenjiamen,CN,29.9576,122.2980,59,
239,rawson,AR,-43.3002,-65.1023,25,
240,tikaitnagar,IN,26.9500,81.5833,59,
305,kingman,US,35.1894,-114.0530,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
import requests

# Set parameters to search for a hotel
radius = 10000  # Radius in meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key  # Only get the first result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
    # Convert the API response to JSON format
    try:
        # name_address = response.json()
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
banda - nearest hotel: #acnindiafy21
shenjiamen - nearest hotel: Zhonghan Hotel
rawson - nearest hotel: Hotel Deportivo
tikaitnagar - nearest hotel: No hotel found
kingman - nearest hotel: Home2 Suites by Hilton Kingman
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
yulara - nearest hotel: Outback Hotel
saint-francois - nearest hotel: Hôtel Epona


,City,Country,Lat,Lng,Humidity,Hotel Name
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
97,shenjiamen,CN,29.9576,122.2980,59,Zhonghan Hotel
239,rawson,AR,-43.3002,-65.1023,25,Hotel Deportivo
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
305,kingman,US,35.1894,-114.0530,49,Home2 Suites by Hilton Kingman
340,santa rosalia,MX,27.3167,-112.2833,56,Sol y Mar
363,narwar,IN,25.6500,77.9000,55,No hotel found
380,yulara,AU,-25.2406,130.9889,57,Outback Hotel
391,saint-francois,FR,46.4154,3.9054,57,Hôtel Epona


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

city_map = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='City',
    alpha=0.7,
    tiles='OSM',
    hover_cols=['City', 'Country', 'Hotel Name'],  # Add hover information
    title="City Map with Hotel Names and Countries in Hover"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)